In [2]:
using DelimitedFiles

A = readdlm("sketches/ww15mgh.grd", skipstart = 1)
A1 = permutedims(A); #transpose into column major order
a1 = vec(A1); #concatenate into a vector to remove unwanted string elements
a1_clean = filter(x -> !(x isa AbstractString), a1);
a_f32 = convert(Array{Float32, 1}, a1_clean) #cast to Float64
#we know that each sequence of 1441 elements corresponds to a constant-latitude
#set of values. after reshaping, each column is a constant-latitude set of
#values running over lon [-pi, pi]
A_F32 = reshape(a_f32, 1441, :) 
A_final = permutedims(A_F32) #transpose so that each row corresponds to a latitude value
open("ww15mgh.bin", "w") do f #opens the specified file, returns it as f, and applies to it the function in the block 
    write(f, htol(A_final))
end
A_retrieved = Matrix{Float32}(undef, 721, 1441)
open("ww15mgh.bin", "r") do f
    ltoh(read!(f, A_retrieved))
end

#ADD HASH TO FILE TO ensure integrity, then load as
const EGM96_geoid_offset = Matrix{Float32}(undef, 721, 1441)
open("ww15mgh.bin", "r") do f
    ltoh(read!(f, EGM96_geoid_offset))
end

@assert all(A_retrieved[1,:] .== 13.606)
@assert all(A_retrieved[end,:] .== -29.534)

1038961-element Vector{Float64}:
  13.606
  13.606
  13.606
  13.606
  13.606
  13.606
  13.606
  13.606
  13.606
  13.606
   ⋮
 -29.534
 -29.534
 -29.534
 -29.534
 -29.534
 -29.534
 -29.534
 -29.534
 -29.534

In [12]:
B = []